In [1]:
# Install roboflow
!pip install roboflow

# Import library
from roboflow import Roboflow
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
# Koneksi ke akun Roboflow
rf = Roboflow(api_key="zFQnSt2437Ig6aCAtSZ4")
project = rf.workspace("yulia-pratiwi-45vut").project("batik-multilabel-big")
version = project.version(29)

# Download dataset (akan tersimpan lokal otomatis)
dataset = version.download("multiclass")  # bisa juga "yolov8", "coco", dll

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to batik-multilabel-big-29 in multiclass:: 100%|██████████| 4118/4118 [00:00<00:00, 4421.92it/s]


In [6]:
# Path folder hasil download
DATA_DIR = dataset.location
ANNOTATION_PATH = os.path.join(DATA_DIR, "train", "_classes.csv")

# Baca anotasi CSV
df = pd.read_csv(ANNOTATION_PATH)

# Deteksi kolom label
label_columns = df.columns[1:]  # semua kolom kecuali 'filename'

# Buat kolom list label untuk setiap gambar
df['labels'] = df[label_columns].apply(lambda row: [label for label, val in row.items() if val == 1], axis=1)
df['file_path'] = df['filename'].apply(lambda x: os.path.join(DATA_DIR, 'train', x))

# Ambil hanya kolom yang dibutuhkan
df_grouped = df[['filename', 'file_path', 'labels']]

# Cek hasil
df_grouped.head()


,filename,file_path,labels
0,IMG_6937_jpeg_jpg.rf.75ac2df3da00a90f9cc629dd6...,/content/batik-multilabel-big-29/train/IMG_693...,"[Kaligrafi, Rembulan]"
1,IMG_20250107_143736_jpg.rf.ee0ce424759b275386c...,/content/batik-multilabel-big-29/train/IMG_202...,"[Kaligrafi, Rembulan]"
2,IMG_6748_jpeg_jpg.rf.de33c5cf8fa8bc3337ee4c241...,/content/batik-multilabel-big-29/train/IMG_674...,"[Kaligrafi, Kembang Cempaka, Kembang Cengkeh]"
3,20241030_153450_jpg.rf.52570b3ada61fcb1e6dc535...,/content/batik-multilabel-big-29/train/2024103...,[Rafflesia]
4,20241030_153454_jpg.rf.15a0b42af4c69ca32535688...,/content/batik-multilabel-big-29/train/2024103...,[Rafflesia]


In [7]:
# Split data 70:20:10
train_val, test = train_test_split(df_grouped, test_size=0.10, random_state=42)
train, val = train_test_split(train_val, test_size=2/9, random_state=42)

print(f"Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")


Train: 2879 | Val: 823 | Test: 412


In [16]:
# Baca CSV
df = pd.read_csv(ANNOTATION_PATH)

# Deteksi label kolom (kecuali filename)
label_columns = df.columns[1:]
df['labels'] = df[label_columns].apply(lambda row: [label for label, val in row.items() if val == 1], axis=1)
df['file_path'] = df['filename'].apply(lambda x: os.path.join(DATA_DIR, 'train', x))

# Kolom akhir yang digunakan
df_grouped = df[['filename', 'file_path', 'labels']]


In [17]:
# Split data
train_val, test = train_test_split(df_grouped, test_size=0.10, random_state=42)
train, val = train_test_split(train_val, test_size=2/9, random_state=42)

print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")


Train: 2879, Val: 823, Test: 412


In [18]:
def label_distribution(df_split, name):
    counts = {label: 0 for label in label_columns}
    for labels in df_split['labels']:
        for l in labels:
            counts[l] += 1
    print(f"\n📊 Distribusi label di {name} set:")
    for label, count in counts.items():
        print(f"  {label:<20}: {count}")

label_distribution(train, "Train")
label_distribution(val, "Validation")
label_distribution(test, "Test")



📊 Distribusi label di Train set:
  Burung Kuau         : 690
  Burung Punai        : 335
  Kaligrafi           : 2124
  Kaligrafi Lawas     : 382
  Kembang Cempaka     : 365
  Kembang Cengkeh     : 365
  Kembang Melati      : 371
  Pohon Hayat         : 348
  Rafflesia           : 373
  Relung Paku         : 335
  Rembulan            : 363

📊 Distribusi label di Validation set:
  Burung Kuau         : 204
  Burung Punai        : 123
  Kaligrafi           : 627
  Kaligrafi Lawas     : 90
  Kembang Cempaka     : 98
  Kembang Cengkeh     : 98
  Kembang Melati      : 96
  Pohon Hayat         : 97
  Rafflesia           : 106
  Relung Paku         : 123
  Rembulan            : 106

📊 Distribusi label di Test set:
  Burung Kuau         : 107
  Burung Punai        : 43
  Kaligrafi           : 313
  Kaligrafi Lawas     : 52
  Kembang Cempaka     : 57
  Kembang Cengkeh     : 57
  Kembang Melati      : 54
  Pohon Hayat         : 55
  Rafflesia           : 47
  Relung Paku         : 43
  Rembulan

In [21]:
# Buat folder baru
BASE_DIR = "batik_multilabel_split2"
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(BASE_DIR, split, "images"), exist_ok=True)

def save_split_data_multihot(df_split, split_name):
    image_dir = os.path.join(BASE_DIR, split_name, "images")
    csv_path = os.path.join(BASE_DIR, split_name, f"{split_name}_labels.csv")

    # Salin gambar
    for _, row in df_split.iterrows():
        src = row['file_path']
        dst = os.path.join(image_dir, os.path.basename(row['file_path']))
        shutil.copy(src, dst)

    # Buat multi-hot DataFrame
    multi_hot_df = pd.DataFrame(0, index=df_split.index, columns=label_columns)
    for idx, labels in enumerate(df_split['labels']):
        for label in labels:
            multi_hot_df.loc[df_split.index[idx], label] = 1
    multi_hot_df.insert(0, 'filename', df_split['filename'].values)

    # Simpan CSV
    multi_hot_df.to_csv(csv_path, index=False)

save_split_data_multihot(train, 'train')
save_split_data_multihot(val, 'val')
save_split_data_multihot(test, 'test')


In [22]:
# Buat file zip
zip_output = f"{BASE_DIR}.zip"
with zipfile.ZipFile(zip_output, 'w') as zipf:
    for root, _, files in os.walk(BASE_DIR):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, BASE_DIR)
            zipf.write(file_path, arcname)

print(f"✅ ZIP berhasil dibuat: {zip_output}")


✅ ZIP berhasil dibuat: batik_multilabel_split2.zip


In [23]:
print("Jumlah gambar:")
print(f"Train      : {len(train)}")
print(f"Validation : {len(val)}")
print(f"Test       : {len(test)}")

total = len(train) + len(val) + len(test)
print(f"\nTotal      : {total}")

print("\nPersentase:")
print(f"Train      : {len(train)/total*100:.2f}%")
print(f"Validation : {len(val)/total*100:.2f}%")
print(f"Test       : {len(test)/total*100:.2f}%")


Jumlah gambar:
Train      : 2879
Validation : 823
Test       : 412

Total      : 4114

Persentase:
Train      : 69.98%
Validation : 20.00%
Test       : 10.01%
